In [8]:
!pip3 install category_encoders

  Using cached category_encoders-2.5.1.post0-py2.py3-none-any.whl (72 kB)
  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 26.2 MB/s eta 0:00:0000:0100:01


In [10]:
!pip3 install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s eta 0:00:0000:0100:01


In [11]:
import numpy as np
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [12]:
#MySQL 연동
conn=pymysql.connect(
    user="yeji",
    passwd='1234',
    host='localhost',
    db='section3'
)

In [23]:
#db 불러오기
movies=pd.read_sql_query('select * from movies',conn)
details=pd.read_sql_query('select * from details',conn)
reviews=pd.read_sql_query('select * from reviews',conn)

#모델링에 사용할 데이터프레임 병합
df=pd.merge(reviews,details,on='movie_id',how='left')

#유저가 5점 만점 중 3점 이상의 점수를 부여했는지를 새로운 컬럼으로 추가 (타겟)
three=[]
for i in df.index:
  if df.loc[i,'user_rating']>=3:
    three.append(1)
  else:
    three.append(0)

df['three']=three

/var/folders/zc/ppbm_g5932z1xjjsjqhpwn6h0000gn/T/ipykernel_77094/996183696.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies=pd.read_sql_query('select * from movies',conn)
/var/folders/zc/ppbm_g5932z1xjjsjqhpwn6h0000gn/T/ipykernel_77094/996183696.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  details=pd.read_sql_query('select * from details',conn)
/var/folders/zc/ppbm_g5932z1xjjsjqhpwn6h0000gn/T/ipykernel_77094/996183696.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews=pd.read_sql_query('select * fr

In [24]:
df.head()

,user_id,movie_id,user_rating,year,rating,running_time,country,age_limit,genre,director,netflix,wavve,tving,disney,watcha,coupang,naver,three
0,1002,62292,5.0,2018,91.78,#117분,#미국,#청소년관람불가,#코미디,데이빗 레이치,0,0,0,1,0,0,0,1
1,1004,1540,3.0,2018,90.30,#137분,#한국,#12세이상관람가,#드라마,윤종빈,0,0,1,0,1,1,1,1
2,1004,62199,3.5,2018,87.94,#115분,#한국,#15세이상관람가,#코미디,이재규,0,0,1,0,1,1,1,1
3,1004,62658,4.0,2018,95.74,#117분,#미국,#12세이상관람가,#액션,피터 램지,1,0,0,1,1,0,0,1
4,1004,64253,2.5,2018,59.00,#134분,#미국,#영국,#12세이상관람가,데이빗 예이츠,1,1,0,0,0,1,0,0


In [31]:
#해시태그 제거
df['running_time']=df['running_time'].str.replace('#','')
df['country']=df['country'].str.replace('#','')
df['age_limit']=df['age_limit'].str.replace('#','')
df['genre']=df['genre'].str.replace('#','')
df.head()

,user_id,movie_id,user_rating,year,rating,running_time,country,age_limit,genre,director,netflix,wavve,tving,disney,watcha,coupang,naver,three
0,1002,62292,5.0,2018,91.78,117분,미국,청소년관람불가,코미디,데이빗 레이치,0,0,0,1,0,0,0,1
1,1004,1540,3.0,2018,90.30,137분,한국,12세이상관람가,드라마,윤종빈,0,0,1,0,1,1,1,1
2,1004,62199,3.5,2018,87.94,115분,한국,15세이상관람가,코미디,이재규,0,0,1,0,1,1,1,1
3,1004,62658,4.0,2018,95.74,117분,미국,12세이상관람가,액션,피터 램지,1,0,0,1,1,0,0,1
4,1004,64253,2.5,2018,59.00,134분,미국,영국,12세이상관람가,데이빗 예이츠,1,1,0,0,0,1,0,0


In [85]:
#상영시간 구간화
time_unit=[]
for i in df.index:
  if int(df.loc[i,'running_time'])<60:
    time_unit.append(0)
  elif int(df.loc[i,'running_time'])<120:
    time_unit.append(1)
  else:
    time_unit.append(2)

df['time_unit']=time_unit
df.head()

,user_id,movie_id,user_rating,year,rating,running_time,country,age_limit,genre,director,netflix,wavve,tving,disney,watcha,coupang,naver,three,time_unit
0,1002,62292,5.0,2018,91.78,117,미국,청소년관람불가,코미디,데이빗 레이치,0,0,0,1,0,0,0,1,1
1,1004,1540,3.0,2018,90.30,137,한국,12세이상관람가,드라마,윤종빈,0,0,1,0,1,1,1,1,2
2,1004,62199,3.5,2018,87.94,115,한국,15세이상관람가,코미디,이재규,0,0,1,0,1,1,1,1,1
3,1004,62658,4.0,2018,95.74,117,미국,12세이상관람가,액션,피터 램지,1,0,0,1,1,0,0,1,1
4,1004,64253,2.5,2018,59.00,134,미국,12세이상관람가,미정,데이빗 예이츠,1,1,0,0,0,1,0,0,2


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1436 entries, 0 to 1441
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       1436 non-null   int64  
 1   movie_id      1436 non-null   int64  
 2   user_rating   1436 non-null   float64
 3   year          1436 non-null   object 
 4   rating        1436 non-null   float64
 5   running_time  1436 non-null   object 
 6   country       1436 non-null   object 
 7   age_limit     1436 non-null   object 
 8   genre         1436 non-null   object 
 9   director      1436 non-null   object 
 10  netflix       1436 non-null   int64  
 11  wavve         1436 non-null   int64  
 12  tving         1436 non-null   int64  
 13  disney        1436 non-null   int64  
 14  watcha        1436 non-null   int64  
 15  coupang       1436 non-null   int64  
 16  naver         1436 non-null   int64  
 17  three         1436 non-null   int64  
 18  time_unit     1436 non-null 

In [86]:
#학습/평가 데이터셋 나누기
target='three'
features=['user_id','rating','time_unit','country','age_limit','genre','director']

X_train,X_test,y_train,y_test=train_test_split(df[features],df[target],test_size=0.2,stratify=df[target])

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(1148, 7) (1148,)
(288, 7) (288,)


In [87]:
#기준모델 설정
base=y_train.mode()[0]
baseline=[base]*len(y_train)

print(classification_report(y_train,baseline))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       304
           1       0.74      1.00      0.85       844

    accuracy                           0.74      1148
   macro avg       0.37      0.50      0.42      1148
weighted avg       0.54      0.74      0.62      1148



/opt/anaconda3/envs/project3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/project3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/project3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
#모델링 및 학습
model=make_pipeline(
    OrdinalEncoder(),
    XGBClassifier(
        objective='binary:logistic',
        eval_metric='error',
        n_estimators=200,
        n_jobs=-1,
        learning_rate=0.1,
        use_label_encoder=False,
        colsample_bytree=0.8,
        max_depth=6
    )
)

model.fit(X_train,y_train)

/opt/anaconda3/envs/project3/lib/python3.10/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['country', 'age_limit', 'genre',
                                      'director'],
                                mapping=[{'col': 'country',
                                          'data_type': dtype('O'),
                                          'mapping': 미국           1
호주           2
한국           3
일본           4
영국           5
부탄           6
프랑스          7
독일           8
인도           9
스웨덴         10
브라질         11
스위스         12
스페인         13
콜롬비아        14
태국          15
중국          16
대만          17
핀란드         18
헝가리         19
아일랜드        20
인도네시아       21
러시아         22
아이슬란드       23
캐나다         24
홍콩          25
덴마크         26
네덜란드        27
남아프리카공화국    28
이탈리아        29
그리스         30
멕시코         31
오스트리아       32
NaN         -2
dtype: int64},
                                         {'col': 'age_limit',
                                          'da...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', learning_rate=0.1,
                               max_bin=256, max_cat_threshold=64,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=200, n_jobs=-1, num_parallel_tree=1,
                               predictor='auto', random_state=0, ...))])

In [89]:
#평가 정확도 확인
print('평가 정확도: ',model.score(X_test,y_test))

y_pred=model.predict(X_test)
print(classification_report(y_test,y_pred))

평가 정확도:  0.7743055555555556
              precision    recall  f1-score   support

           0       0.62      0.38      0.47        76
           1       0.80      0.92      0.86       212

    accuracy                           0.77       288
   macro avg       0.71      0.65      0.66       288
weighted avg       0.76      0.77      0.75       288



In [90]:
#모델 피클링
with open('model.pkl','wb') as pickle_file:
  pickle.dump(model,pickle_file)